In [20]:
import os
import sys
import json
import pickle
import itertools
from pprint import pprint

import requests
from bs4 import BeautifulSoup

# BASEURL = 'http://www.tylervigen.com/page?page='
BASEURL = 'http://tylervigen.com/view_correlation?id='

soupstash = '../vigenstash/'

def soup_from_stash_or_url(page):
    
    stashpath = getstashpath(page)
    cool = True
    
    if os.path.isfile(stashpath):
        # print 'unstashing page %d from file' % page
        with open(stashpath,'r') as htmlfile:
            html = htmlfile.read()
        if len(html) < 10:
            # print 'WARNING: page %d is a dud' % (page)
            cool = False

    else:
        url = BASEURL + str(page)
        if not page%100:
            print 'retrieving page %d from url: %s' % (page,url)
        r = requests.get(url)
        
        if r.ok:
            html = r.content
            if ismissing(html):
                cool = False
                print 'WARNING: page %d is a dud. %s' % (page,url)
                html = 'dud.'
                
            with open(stashpath,'w') as htmlfile:
                htmlfile.write(html)
        else: 
            print 'WARNING: could not retrieve page %d: %s' % (page,url)
            cool = False
    
    if not(page == 1) and isdupe(html):
        print 'dupe.'
        cool = false
    
    if not cool: 
        return cool
    else:
        soup = BeautifulSoup(html,'html5lib')
        return soup

def isdupe(html):
    p1_stash = getstashpath(1)
    with open (p1_stash,'r') as stashfile:
        p1 = stashfile.read()
    return p1 == html

def ismissing(html):
    with open(soupstash + 'vigen_missing.html','r') as missfile:
        missing_html = missfile.read()
    return html == missing_html
    
def getstashpath(page):
    stashpath = soupstash + 'vigen_page_' + str(page) + '.html'
    return stashpath

def getsoup(url):
    r = requests.get(url)
    if not r.ok: 
        return False
    return BeautifulSoup(r.content,'html5lib')

def flatten(listolists):
    return list(itertools.chain.from_iterable(listolists))

def tendudsinarow(duds):
    return ( sum(duds[-10:]) == (duds[-1]*10 - 45) )

def dudsinarow(duds,n):
    return ( sum(duds[(-1*n):]) == (duds[-1]*n - sum(range(n))))

In [22]:
page = 1
cool = True
things = []
pairs = {}
duds = []
dud_limit = 100

while cool:

    if page > dud_limit and dudsinarow(duds,dud_limit):
        print "Stopping due to %d-dud deluge. Thank you and have a nice day" % dud_limit
        cool = False
        continue
    
    try:
        soup = soup_from_stash_or_url(page)
    except requests.exceptions.RequestException:
        print 'failure retrieving page %d: %s' % (page, (BASEURL + str(page)))
        continue
    page += 1

    if not soup: 
        duds.append(page-1)
        continue
        
    try:
        items, r = correlates_from_soup(soup)
    except IndexError:
        duds.append(page-1)
        continue
        
    things += list(items)
    pairs[items] = r
    
    if not page%500:
        results = {'things':things,'pairs':pairs,'duds':duds}
        with open('../vigenstash/pickle_vigen_latest.pkl','w') as outfile:
            pickle.dump(results,outfile)
    
print page
print len(things)



retrieving page 3500 from url: http://tylervigen.com/view_correlation?id=3500
retrieving page 3600 from url: http://tylervigen.com/view_correlation?id=3600
retrieving page 3700 from url: http://tylervigen.com/view_correlation?id=3700
retrieving page 3800 from url: http://tylervigen.com/view_correlation?id=3800
retrieving page 3900 from url: http://tylervigen.com/view_correlation?id=3900
retrieving page 4000 from url: http://tylervigen.com/view_correlation?id=4000
retrieving page 4100 from url: http://tylervigen.com/view_correlation?id=4100
retrieving page 4200 from url: http://tylervigen.com/view_correlation?id=4200
failure retrieving page 4215: http://tylervigen.com/view_correlation?id=4215
retrieving page 4300 from url: http://tylervigen.com/view_correlation?id=4300
retrieving page 4400 from url: http://tylervigen.com/view_correlation?id=4400
retrieving page 4500 from url: http://tylervigen.com/view_correlation?id=4500
failure retrieving page 4597: http://tylervigen.com/view_correlat

KeyboardInterrupt: 

In [10]:
def correlates_from_soup(soup):
    '''For old-style pages (the kind that work but have 1 comparison per page)
    '''
    links = soup.find_all('a',class_='variable')
    items = [k.text for k in links]
    corr = float(soup.find_all('b')[-1].text.split(':')[1].strip())
    return tuple(items), corr

def sources_from_soup(soup):
    '''For new-style pages (the kind that don't iterate past 1)
    '''
    source_divs = soup.find_all('div',class_='sources')
    sources = [src.find_all('a') for src in source_divs]
    sources = [src['href'] for src in flatten(sources)]
    # sources = list(itertools.chain.from_iterable(sources))
    
    return sources


def figure_scripts_from_soup(soup):
    '''For new-style pages (the kind that don't iterate past 1)
    '''
    javascripts = soup.find_all('script',{'type':'text/javascript'})
    return [s for s in javascripts if s.text]

def items_from_script(script):
    '''For new-style pages (the kind that don't iterate past 1)
    '''
    script_text = script.text
    cleaned_lines = [s.strip() for s in script_text.split('\n') if s.strip()] 
    script = ''.join(cleaned_lines)

    title_html = script.split('title: {text: \'')[1].split(',style:')[0]
    item_spans = BeautifulSoup(title_html).find_all('span')
    items = [s.text for s in item_spans if 'correlates with' not in s.text.lower()]
    
    return items


In [228]:
## Book sources page scrape 

soup = getsoup('http://www.tylervigen.com/sources')
h2s = soup.find_all('h2')
headers = [h2.text for h2 in h2s]
uniques = [text for text in headers if text.startswith('Page')]
pages = [int(u.split(':')[0].split(' ')[1]) for u in uniques]
print '# pages: %d' % len(pages)

ref_lists = soup.find('div',class_='content').find_all('ul',recursive=False)[1:]
comparisons = [u.split(':')[1].strip().split('and') for u in uniques if len(u.split(':')) > 1]
print '\ncomparisons: \n%s [...]' % repr(comparisons[:5])

import itertools 

compared_things = [t.strip().lower() for t in list(itertools.chain.from_iterable(comparisons))]

print '\n# total compared things: %d' % len(compared_things)
print '\n# unique compared things: %d' % len(set(compared_things))
print '\ncompared things: \n%s [...]' % repr(sorted(list(set(compared_things)))[1:10])
#print sorted(compared_things)

# pages: 90

comparisons: 
[[u'Hot dogs ', u' fiction books'], [u'Cheese ', u' bedsheet deaths'], [u'Margarine ', u' Maine divorces'], [u'Chicken ', u' paper'], [u"McDonald's satisfaction ", u' deaths by choking']] [...]

# total compared things: 175

# unique compared things: 172

compared things: 
[u'airline complaints', u'alcohol', u'american idol', u'argentina', u'bankruptcy', u'basketball', u'bedsheet deaths', u'beef lighting', u'bees'] [...]


In [8]:
import math
import scipy

def tfromr(r,N): 
    return (r/math.sqrt( (1-r**2)/(N-2) ))

def pfromr(r,N): 
    return (2*(1 - scipy.stats.t.cdf(tfromr(r,N),N-2)))


In [21]:
print len(pairs)

3221
